In [15]:
import os,datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path
'''
SEED_XML_DIR = '/data/darknet/python/xml/'
SEED_IMG_DIR = '/data/darknet/python/image/'
SPEC_OBJ_DIR = '../manufactur/special_obj/'
GENE_IMG_WITH_SPECIAL_OBJ = '../generated/img_with_special_obj/'
GENE_XML_WITH_SPECIAL_OBJ = '../generated/xml_with_special_obj/'
'''
SEED_IMG_DIR = '/data/wxjs/JPEGImages/'
SEED_XML_DIR = '/data/wxjs/Changed_Annotations/'
SPEC_OBJ_DIR = '/data/wxjs/generated/img_objects/'
GENE_IMG_WITH_SPECIAL_OBJ = '/data/wxjs/generated/img_with_special_obj/'
GENE_XML_WITH_SPECIAL_OBJ = '/data/wxjs/generated/xml_with_special_obj/'

def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2]) 
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)

def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
def get_bboxes_from_etree(etree):
    root = tree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0.005):
            return False
        continue
    return True

def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree

def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img

def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    print(new_bbox)
    bboxes = get_bboxes_from_etree(etree)
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
        print(new_bbox)
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, etree)
    
    return new_pil_img, new_xml_etree

def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object

def generate_xml_info_under_object(attribut_vale_dic):
    element_object = ET.Element('object')
    if(len(attribut_vale_dic) == 0):
        return False
    for key in attribut_vale_dic.keys():
        tag_name = ET.SubElement(element_object, key)
        tag_name.text = attribut_vale_dic[key]
    return element_object
#generate_xml_info_under_object({'name':'my-class','difficult':'0'})

In [5]:
'''
seed_xml_names = os.listdir(SEED_XML_DIR)
special_obj_cls = os.listdir(SPEC_OBJ_DIR)
#classes = ['person', 'bicycle', 'bottle', 'cup']
classes = ['slagcar_00']
for xml_name in seed_xml_names:
    img_name = SEED_IMG_DIR + xml_name[:-3] + 'jpg'
    xml_name = SEED_XML_DIR + xml_name
    print(img_name)
    print(xml_name)
    tree=ET.parse(xml_name)
    img = cv2.imread(img_name)
    for cls in special_obj_cls:
        if(not cls in classes):
            continue
        objs = os.listdir(SPEC_OBJ_DIR + cls)
        for obj in objs:
            print(obj)
            if(not (obj.endswith('.png') or obj.endswith('.jpg'))):
                continue
            img_obj = cv2.imread(SPEC_OBJ_DIR + cls + '/' + obj)
            dic_ = {'name':cls, 'difficult':'0'}
            obj_element = generate_xml_info_under_object(dic_)
            new_pil_img, new_xml_etree = inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree)
            print('insert success')
            tree = new_xml_etree
            img = np.array(new_pil_img)
    ## save xml and img
    print('save image and xml')
    time_mark = datetime.datetime.now()
    time_str = time_mark.strftime("%Y%m%d%H%M%S_")
    new_pil_img = Image.fromarray(img[:,:,(2, 1, 0)])
    new_pil_img.save(GENE_IMG_WITH_SPECIAL_OBJ + time_str + img_name.split('/')[-1])
    tree.write(GENE_XML_WITH_SPECIAL_OBJ + time_str + xml_name.split('/')[-1])
    #break
'''

'\nseed_xml_names = os.listdir(SEED_XML_DIR)\nspecial_obj_cls = os.listdir(SPEC_OBJ_DIR)\n#classes = [\'person\', \'bicycle\', \'bottle\', \'cup\']\nclasses = [\'slagcar_00\']\nfor xml_name in seed_xml_names:\n    img_name = SEED_IMG_DIR + xml_name[:-3] + \'jpg\'\n    xml_name = SEED_XML_DIR + xml_name\n    print(img_name)\n    print(xml_name)\n    tree=ET.parse(xml_name)\n    img = cv2.imread(img_name)\n    for cls in special_obj_cls:\n        if(not cls in classes):\n            continue\n        objs = os.listdir(SPEC_OBJ_DIR + cls)\n        for obj in objs:\n            print(obj)\n            if(not (obj.endswith(\'.png\') or obj.endswith(\'.jpg\'))):\n                continue\n            img_obj = cv2.imread(SPEC_OBJ_DIR + cls + \'/\' + obj)\n            dic_ = {\'name\':cls, \'difficult\':\'0\'}\n            obj_element = generate_xml_info_under_object(dic_)\n            new_pil_img, new_xml_etree = inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree)\n            pri

In [ ]:
seed_xml_names = os.listdir(SEED_XML_DIR)
special_obj_cls = os.listdir(SPEC_OBJ_DIR)

classes = ['slagcar_00']

for cls in special_obj_cls:
        if(not cls in classes):
            continue
            
        objs = os.listdir(SPEC_OBJ_DIR + cls)
        for obj in objs:
            print(obj)
            if(not (obj.endswith('.png') or obj.endswith('.jpg'))):
                continue

            for xml_name in seed_xml_names:
                img_name = SEED_IMG_DIR + xml_name[:-3] + 'jpg'
                xml_name = SEED_XML_DIR + xml_name
                print(img_name)
                print(xml_name)
                tree=ET.parse(xml_name)
                img = cv2.imread(img_name)
                img_obj = cv2.imread(SPEC_OBJ_DIR + cls + '/' + obj)
                if((img.shape[0] < img_obj.shape[0]) or (img.shape[1] < img_obj.shape[1]) or 
                   ((img.shape[0]*img.shape[1])/(img_obj.shape[0]*img_obj.shape[1]) < 4)):
                    continue
                dic_ = {'name':cls, 'difficult':'0'}
                obj_element = generate_xml_info_under_object(dic_)
                new_pil_img, new_xml_etree = inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree)
                print('insert success')

                ## save xml and img
                print('save image and xml')
                time_mark = datetime.datetime.now()
                time_str = time_mark.strftime("%Y%m%d%H%M%S_")
                new_pil_img = Image.fromarray(img[:,:,(2, 1, 0)])
                new_pil_img.save(GENE_IMG_WITH_SPECIAL_OBJ + time_str + img_name.split('/')[-1])
                tree.write(GENE_XML_WITH_SPECIAL_OBJ + time_str + xml_name.split('/')[-1])
                #break

00acecb11ebad9b1c3647977e15b472b_0ps.jpg
/data/wxjs/JPEGImages/01c08702b7f7f562547e2a8718c8ab07.jpg
/data/wxjs/Changed_Annotations/01c08702b7f7f562547e2a8718c8ab07.xml
[2562, 3181, 861, 1747]
insert success
save image and xml
/data/wxjs/JPEGImages/022014a454e80e353bef2690ea813d87.jpg
/data/wxjs/Changed_Annotations/022014a454e80e353bef2690ea813d87.xml
[1156, 1775, 991, 1877]
new_bbox not suitable, retry...
[367, 986, 135, 1021]
insert success
save image and xml
/data/wxjs/JPEGImages/014f328da8e0a69c217fc10635c3f7a8.jpg
/data/wxjs/Changed_Annotations/014f328da8e0a69c217fc10635c3f7a8.xml
[1218, 1837, 673, 1559]
insert success
save image and xml
/data/wxjs/JPEGImages/0127d83c870c04c6db0682ed6dd27162.jpg
/data/wxjs/Changed_Annotations/0127d83c870c04c6db0682ed6dd27162.xml
[777, 1396, 465, 1351]
new_bbox not suitable, retry...
[914, 1533, 147, 1033]
new_bbox not suitable, retry...
[2250, 2869, 893, 1779]
insert success
save image and xml
/data/wxjs/JPEGImages/01dce76be1b050dcf3f1014cd670acfc.

/data/wxjs/JPEGImages/0008ec851e66864a9ad38161f7e9d5a8.jpg
/data/wxjs/Changed_Annotations/0008ec851e66864a9ad38161f7e9d5a8.xml
[371, 990, 310, 1196]
insert success
save image and xml
/data/wxjs/JPEGImages/0102e8b3e006c5b0146ec15cb4ea95a0.jpg
/data/wxjs/Changed_Annotations/0102e8b3e006c5b0146ec15cb4ea95a0.xml
[12, 631, 460, 1346]
insert success
save image and xml
/data/wxjs/JPEGImages/021c507164d4d5e37cbae8cbac3199fd.jpg
/data/wxjs/Changed_Annotations/021c507164d4d5e37cbae8cbac3199fd.xml
[1948, 2567, 966, 1852]
insert success
save image and xml
/data/wxjs/JPEGImages/0190b4cb574472b6f9f5c0628f4cdd5c.jpg
/data/wxjs/Changed_Annotations/0190b4cb574472b6f9f5c0628f4cdd5c.xml
[2567, 3186, 384, 1270]
insert success
save image and xml
/data/wxjs/JPEGImages/02acfbe1bcbb24461b3550c8bcb2d9d7.jpg
/data/wxjs/Changed_Annotations/02acfbe1bcbb24461b3550c8bcb2d9d7.xml
[1296, 1915, 443, 1329]
insert success
save image and xml
/data/wxjs/JPEGImages/00046e49559722ee8e8159593d028888.jpg
/data/wxjs/Changed_An

/data/wxjs/JPEGImages/0221579b3252d9bebb81798cf8543809.jpg
/data/wxjs/Changed_Annotations/0221579b3252d9bebb81798cf8543809.xml
[1212, 1831, 158, 1044]
new_bbox not suitable, retry...
[95, 714, 95, 981]
insert success
save image and xml
/data/wxjs/JPEGImages/022d0841346351c4c8f767242d6d5456.jpg
/data/wxjs/Changed_Annotations/022d0841346351c4c8f767242d6d5456.xml
[1705, 2324, 30, 916]
insert success
save image and xml
/data/wxjs/JPEGImages/01017b4f5dc5d635199d9670afd949fe.jpg
/data/wxjs/Changed_Annotations/01017b4f5dc5d635199d9670afd949fe.xml
[2144, 2763, 691, 1577]
insert success
save image and xml
/data/wxjs/JPEGImages/01c672bdb18dc60de44be911d0f5a6a2.jpg
/data/wxjs/Changed_Annotations/01c672bdb18dc60de44be911d0f5a6a2.xml
[172, 791, 640, 1526]
insert success
save image and xml
/data/wxjs/JPEGImages/025017ce46d063cb1f5db57fe2e48990.jpg
/data/wxjs/Changed_Annotations/025017ce46d063cb1f5db57fe2e48990.xml
[551, 1170, 139, 1025]
new_bbox not suitable, retry...
[952, 1571, 403, 1289]
new_bbox

/data/wxjs/JPEGImages/017b7fbbb261669feae86d3187d774ae.jpg
/data/wxjs/Changed_Annotations/017b7fbbb261669feae86d3187d774ae.xml
[1593, 2253, 982, 1985]
new_bbox not suitable, retry...
[385, 1045, 722, 1725]
insert success
save image and xml
/data/wxjs/JPEGImages/0273f116e6f887b524893d339d296ad3.jpg
/data/wxjs/Changed_Annotations/0273f116e6f887b524893d339d296ad3.xml
[389, 1049, 328, 1331]
insert success
save image and xml
/data/wxjs/JPEGImages/02417750d2945c439841ba60ebcf7ad9.jpg
/data/wxjs/Changed_Annotations/02417750d2945c439841ba60ebcf7ad9.xml
[90, 750, 408, 1411]
insert success
save image and xml
/data/wxjs/JPEGImages/01ce8475f9a2a431556d3fb5bee43aae.jpg
/data/wxjs/Changed_Annotations/01ce8475f9a2a431556d3fb5bee43aae.xml
[2482, 3142, 469, 1472]
insert success
save image and xml
/data/wxjs/JPEGImages/009966ce9c9b2fc621a0ccf501d870c3.jpg
/data/wxjs/Changed_Annotations/009966ce9c9b2fc621a0ccf501d870c3.xml
[1608, 2268, 720, 1723]
insert success
save image and xml
/data/wxjs/JPEGImages/00